<a href="https://colab.research.google.com/github/jvataidee/DataAnalysis/blob/master/Analise_Financeira_Mercado_Gado_de_Corte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**by: [João Ataíde](https://www.joaoataide.com)**
# Análise Mercado Financeira de Gado de Corte


---


## Importanto bibliotecas básicas

In [1]:
#importar bibliotecas básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# configurar a visualização
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## Obtenção dos dados

In [2]:
!pip install investpy -q

     |████████████████████████████████| 4.4 MB 9.0 MB/s 
     |████████████████████████████████| 241 kB 58.1 MB/s 
     |████████████████████████████████| 6.3 MB 31.9 MB/s 
     |████████████████████████████████| 510 kB 48.3 MB/s 


In [3]:
!pip install yfinance -q

In [5]:
#importar bibliotecas financeiras
import investpy
import yfinance as yf

### Dados de Commodities

In [8]:
#Lista de commodities
commodities = ["Live Cattle", "Feeder Cattle", "US Soybeans","US Corn"]

#Data de inicio e fim de análise
inicio = "01/01/2014"
fim = "31/12/2020"

In [10]:
#Função para importar dados das commodities
def commodities_df(commodities, inicio,fim):
  precos = pd.DataFrame()
  for i in commodities:
    df = investpy.commodities.get_commodity_historical_data(commodity=i, from_date=inicio, to_date=fim).Close
    df.rename(i, inplace=True)
    df.columns = [i]
    precos = pd.concat([precos,df], axis=1)
    precos.index.name="Date"
  return precos


#Data Frame de commodites
df = commodities_df(commodities,inicio,fim)

#Importar dollar
doll = yf.Ticker("USDBRL=x")
dollar = pd.DataFrame(doll.history(start ="2014-01-01", end ="2020-12-31" ).Close)
dollar.columns = ["Dollar"]
dollar.index.name="Date"

#Concatenar dado
dados =pd.merge(df, dollar, how='inner', right_index=True, left_index=True)
dados.head()

,Live Cattle,Feeder Cattle,US Soybeans,US Corn,Dollar
Date,,,,,
2014-01-01,134.688,NaN,1289.38,421.38,2.3592
2014-01-02,135.625,167.00,1270.75,420.50,2.3571
2014-01-03,136.300,167.62,1271.75,423.50,2.3847
2014-01-06,136.825,168.03,1278.13,427.75,2.3715
2014-01-07,136.525,168.05,1274.50,426.00,2.3737


In [11]:
#Agrupar a média por trimestre
dados_trimestre = dados.groupby(pd.Grouper(freq='Q')).mean()
dados_trimestre = dados_trimestre.reset_index()
dados_trimestre.head()

,Date,Live Cattle,Feeder Cattle,US Soybeans,US Corn,Dollar
0,2014-03-31,142.532871,171.350492,1351.535000,451.977903,2.361074
1,2014-06-30,142.672619,190.970476,1462.786349,479.190476,2.225935
2,2014-09-30,154.548047,220.610000,1089.502969,359.718750,2.268325
3,2014-12-31,165.997308,233.692187,1008.709385,372.705846,2.538731
4,2015-03-31,157.177049,210.525246,991.899180,384.864754,2.864362


### Dados do SIDRA

In [12]:
import requests

#link do banco de dados sidra
consulta = 'https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/all/p/all/c12716/all/c18/55,56,992,111734,111735/c12529/all/d/v1000151%202,v1000284%202,v1000285%202'
ufs = [5]

#Baixar dados do sidra
df2 = pd.DataFrame()
for uf in ufs:
    url = consulta.format(uf)
    dados_json = requests.get(url).json()
    dados_pandas = pd.DataFrame(dados_json)
    df2 = pd.concat([dados_pandas,df2])
  
#Renomear colunas
sidra = df2.copy() 
colunas = list(df2.iloc[0,:])
sidra.columns = colunas

#Filtrar dados
sidra = sidra.iloc[1:,:]
sidra = sidra[sidra["Variável"]=="Animais abatidos"]
sidra = sidra[sidra["Tipo de inspeção"]=="Total"]
sidra = sidra[sidra["Unidade de Medida"]=="Cabeças"]
sidra = sidra[sidra["Referência temporal"]=="Total do trimestre"]

#Escolher colunas necessárias para a análise
sidra = sidra.reset_index()
sidra = sidra.iloc[:, [5, 11, 15]]

#Dropar coluna de valor X
sidra = sidra.drop(sidra[sidra["Valor"] == 'X'].index)

#tranformar valor em float
sidra["Valor"] = sidra["Valor"].astype(float)

#Encontrar datas
times = sidra.Trimestre.str.split(expand=True)
times[2] = pd.to_datetime(times[2])
sidra3 = sidra.copy()
sidra3 = sidra3[times[2] >= "2014-01-01"]
sidra3 = sidra3[times[2] < "2021-01-01"]
sidra3.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,Valor,Trimestre,Tipo de rebanho bovino
340,3993691.0,1º trimestre 2014,Bois
341,3067441.0,1º trimestre 2014,Vacas
342,8372872.0,1º trimestre 2014,Total
343,447977.0,1º trimestre 2014,Novilhos
344,863763.0,1º trimestre 2014,Novilhas


In [16]:
#Separar dados pelo tipo de rebanho
bois = sidra3[sidra3["Tipo de rebanho bovino"] == "Bois"]
vacas = sidra3[sidra3["Tipo de rebanho bovino"] == "Vacas"]
novilhas = sidra3[sidra3["Tipo de rebanho bovino"] == "Novilhos"]
novilhos = sidra3[sidra3["Tipo de rebanho bovino"] == "Novilhas"]
total = sidra3[sidra3["Tipo de rebanho bovino"] == "Total"]

In [17]:
#Função de contador
def pre_join(dataframe, nome):
  dataframe = dataframe.reset_index()
  dataframe = dataframe.iloc[:, [1]]
  dataframe.columns = [nome]
  return dataframe

#Criar contador de trimestre
bois = pre_join(bois, "Bois")
vacas = pre_join(vacas, "Vacas")
novilhas = pre_join(novilhas, "Novilhas")
novilhos = pre_join(novilhos, "Novilhos")
total = pre_join(total, "Total")

### Juntar os dois datasets

In [18]:
#Função de join
df_final = dados_trimestre.copy()
df_final = df_final.join(bois)
df_final = df_final.join(vacas)
df_final = df_final.join(novilhas)
df_final = df_final.join(novilhos)
df_final = df_final.join(total)

#indexar por data
df_final=df_final.set_index("Date")
df_final.head()

,Live Cattle,Feeder Cattle,US Soybeans,US Corn,Dollar,Bois,Vacas,Novilhas,Novilhos,Total
Date,,,,,,,,,,
2014-03-31,142.532871,171.350492,1351.535000,451.977903,2.361074,3993691.0,3067441.0,447977.0,863763.0,8372872.0
2014-06-30,142.672619,190.970476,1462.786349,479.190476,2.225935,4317627.0,2899909.0,438998.0,882370.0,8538904.0
2014-09-30,154.548047,220.610000,1089.502969,359.718750,2.268325,4627826.0,2561439.0,561864.0,719751.0,8470880.0
2014-12-31,165.997308,233.692187,1008.709385,372.705846,2.538731,4780410.0,2501305.0,565254.0,678093.0,8525062.0
2015-03-31,157.177049,210.525246,991.899180,384.864754,2.864362,3967926.0,2669421.0,363654.0,738649.0,7739650.0


##Análises exploratória

###Pré-processamento


### Análises da série temporal

## Modelo de previsão

### Tratamento dos dados

##Conclusão